In [ ]:
%pip install transformers accelerate torch evaluate datasets peft bitsandbytes

# BLEURT score
- [Link to Repo](https://github.com/google-research/bleurt/blob/master/checkpoints.md)
- [HF Eval Page](https://huggingface.co/spaces/evaluate-metric/bleurt)

In [ ]:
# mount gdrive to save results
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

In [ ]:
%cd /content/drive/MyDrive/WCU_THESIS/AITA_Evaluation

In [ ]:
!pip install --upgrade pip  # ensures that pip is current
!git clone https://github.com/google-research/bleurt.git
%cd /content/drive/MyDrive/WCU_THESIS/AITA_Evaluation/bleurt
!pip install -e .

In [ ]:
# Downloads the BLEURT-base checkpoint.
!wget https://storage.googleapis.com/bleurt-oss-21/BLEURT-20.zip .
!unzip BLEURT-20.zip

In [ ]:
%cd ..

In [ ]:
import json

with open('flanT5_small_evaluation/test_results.json', 'r') as file:
    test_results = json.load(file)

candidates = test_results['predictions']
references = test_results['reference_texts']

In [ ]:
# Open a file to write
with open('flanT5_small_evaluation/test_result_pairs.jsonl', 'w') as file:
    # Iterate over the zipped lists
    for ref, cand in zip(references, candidates):
        # Create a dictionary with the specified keys and values
        json_object = {'reference': ref, 'candidate': cand}
        # Convert the dictionary to a JSON string
        json_str = json.dumps(json_object)
        # Write the JSON string to the file with a newline
        file.write(json_str + '\n')

In [ ]:
import shutil
source_path = '/content/drive/MyDrive/WCU_THESIS/AITA_Evaluation/flanT5_small_evaluation/test_result_pairs.jsonl'
destination_path = '/content/drive/MyDrive/WCU_THESIS/AITA_Evaluation/bleurt/test_result_pairs.jsonl'
shutil.copy2(source_path, destination_path)

In [ ]:
%cd /content/drive/MyDrive/WCU_THESIS/AITA_Evaluation/bleurt

In [ ]:
import json

# Path to your JSONL file

# Create an empty list to store the data
test_result_pairs = []

# Open the file and read line by line
with open('test_result_pairs.jsonl', 'r') as file:
    for line in file:
        # Parse each line as a JSON object and add to the list
        test_result_pairs.append(json.loads(line))


In [ ]:
# create flan_t5_small_scores.txt
!touch flan_t5_small_scores.txt

In [ ]:
!python -m bleurt.score_files \
  -sentence_pairs_file=test_result_pairs.jsonl \
  -bleurt_checkpoint=BLEURT-20 \
  -scores_file=flan_t5_small_scores.txt